# VITK Project

Benjamin Clene
Yanis Farhat

In [24]:
#!pip install -r requirements.txt
#!pip install vtk
!pip install --user itkwidgets

In [2]:
import itk
import os
import matplotlib
import matplotlib.pyplot as plt
#!/usr/bin/env python

import sys
import argparse

import vtk
from ipywidgets import interact, fixed
import ipywidgets as widgets
from IPython.display import display

In [5]:

# Chargement du fichier de l'image volumétrique
filename = "Data/case6_gre1.nrrd"
reader = vtk.vtkNrrdReader()
reader.SetFileName(filename)
reader.Update()

# Obtention des données de l'image
image_data = reader.GetOutput()

# Création du mapper et de l'acteur pour la visualisation
volume_mapper = vtk.vtkSmartVolumeMapper()
volume_mapper.SetInputData(image_data)

volume_property = vtk.vtkVolumeProperty()
volume_property.ShadeOff()
volume_property.SetInterpolationTypeToLinear()

volume = vtk.vtkVolume()
volume.SetMapper(volume_mapper)
volume.SetProperty(volume_property)

# Création de la fenêtre de rendu, du renderer et ajout du volume
renderer = vtk.vtkRenderer()
renderer.AddVolume(volume)

render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

# Création de l'interacteur pour visualiser la fenêtre de rendu
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# Exécution de la visualisation
interactor.Initialize()
render_window.Render()
interactor.Start()


In [27]:
reader.GetPixelTypeAsString()

AttributeError: 'vtkmodules.vtkIOImage.vtkNrrdReader' object has no attribute 'GetPixelTypeAsString'

In [30]:
import itk
from itkwidgets import view

# Chargement du fichier de l'image volumétrique
filename = "Data/case6_gre1.nrrd"
ImageType = itk.Image[itk.SS, 3]
image = itk.imread(filename, pixel_type=ImageType)

# Création du filtre de croissance de région
region_growing_filter = itk.ConfidenceConnectedImageFilter[ImageType, ImageType].New()
region_growing_filter.SetInput(image)
region_growing_filter.SetSeed([x, y, z])  # Coordonnées de la graine de départ pour la segmentation
region_growing_filter.SetNumberOfIterations(5)  # Nombre d'itérations pour la croissance de région
region_growing_filter.SetMultiplier(2.5)  # Multiplieur pour le critère de croissance

# Exécution du filtre de croissance de région
segmented_image = region_growing_filter.GetOutput()
segmented_image.Update()

# Visualisation de l'image segmentée
view(segmented_image, ui_collapsed=True)



TemplateTypeError: itk.Image is not wrapped for input type `itk.Image[itk.SS,3], int`.

To limit the size of the package, only a limited number of
types are available in ITK Python. To print the supported
types, run the following command in your python environment:

    itk.Image.GetTypes()

Possible solutions:
* If you are an application user:
** Convert your input image into a supported format (see below).
** Contact developer to report the issue.
* If you are an application developer, force input images to be
loaded in a supported pixel type.

    e.g.: instance = itk.Image[itk.RGBPixel[itk.UC], int].New(my_input)

* (Advanced) If you are an application developer, build ITK Python yourself and
turned to `ON` the corresponding CMake option to wrap the pixel type or image
dimension you need. When configuring ITK with CMake, you can set
`ITK_WRAP_${type}` (replace ${type} with appropriate pixel type such as
`double`). If you need to support images with 4 or 5 dimensions, you can add
th